# Tiền xử lý dữ liệu - HR Analytics

**Mục tiêu:** Notebook này thực hiện các bước tiền xử lý cần thiết cho bộ dữ liệu HR Analytics, bao gồm xử lý giá trị thiếu và mã hóa đặc trưng. Chúng ta sẽ sử dụng các hàm từ `src/data_processing.py`.

## 1. Giới thiệu và Load Dữ liệu

In [ ]:
# Thêm đường dẫn đến thư mục src
import sys
sys.path.append('../src')

# Import các thư viện và hàm cần thiết
import numpy as np
from data_processing import load_data, get_missing_summary

# Tải dữ liệu
filepath = '../data/raw/aug_train.csv'
data = load_data(filepath)

# Hiển thị tóm tắt giá trị thiếu ban đầu
print("--- Tóm tắt giá trị thiếu ban đầu ---")
missing_summary = get_missing_summary(data)
for col, summary in missing_summary.items():
    print(f"{col}: {summary['missing_count']} ({summary['missing_percentage']:.2f}%)")

## 2. Xử lý Giá trị Thiếu (Missing Values)

Chúng ta sẽ điền các giá trị bị thiếu bằng các chiến lược khác nhau: 'median' cho các cột số và 'mode' cho các cột phân loại.

In [ ]:
# Import hàm cần thiết
from data_processing import impute_missing_values

# Điền giá trị thiếu
for col in data.dtype.names:
    if np.issubdtype(data[col].dtype, np.number):
        data = impute_missing_values(data, col, strategy='median')
    else:
        data = impute_missing_values(data, col, strategy='mode')

# Hiển thị tóm tắt giá trị thiếu sau khi điền
print("--- Tóm tắt giá trị thiếu sau khi điền ---")
missing_summary_after = get_missing_summary(data)
for col, summary in missing_summary_after.items():
    print(f"{col}: {summary['missing_count']} ({summary['missing_percentage']:.2f}%)")

## 3. Mã hóa Đặc trưng Phân loại (Categorical Feature Encoding)

Chúng ta sẽ sử dụng mã hóa one-hot cho các đặc trưng phân loại để chuyển chúng thành dạng số mà các mô hình học máy có thể sử dụng.

In [ ]:
# Import hàm cần thiết
from data_processing import one_hot_encode

# Mã hóa one-hot
categorical_cols = [name for name, dtype in data.dtype.fields.items() if not np.issubdtype(dtype[0], np.number)]
for col in categorical_cols:
    data = one_hot_encode(data, col)

# Hiển thị thông tin sau khi mã hóa
print("--- Dữ liệu sau khi mã hóa ---")
print(data.dtype.names)
print(data[:5])

## 4. Lưu Dữ liệu đã Tiền xử lý

In [ ]:
# Lưu dữ liệu đã xử lý
np.save('../data/processed/processed_train.npy', data)